In [1]:
import requests
import glob
import json

sentiments = {}
for path in glob.glob('./dumps/sentiment/*'):
    filename = path.split('/')[-1]
    identifier = filename.split('.')[0]
    sentiments[identifier] = json.load(open(path))

for path in glob.glob('./dumps/texts/*')[:1000]:
    filename = path.split('/')[-1]
    identifier = filename.split('.')[0]
    with open(path, 'r') as f:
        per_batch = 500
        offset = 0
        words = f.read().split()
        
        chunks = []
        while offset * per_batch < len(words):
            chunks.append({
                'language': 'de',
                'text': ' '.join(words[(offset * per_batch):(offset * per_batch + per_batch)]),
                'id': identifier + '.' + str(offset)
            })
            offset += 1
            
        r = requests.post(
            'https://westus.api.cognitive.microsoft.com/text/analytics/v2.0/keyPhrases',
            headers={
                'Content-Type': 'application/json',
                'Accept': 'application/json',
                'Ocp-Apim-Subscription-Key': 'd1ca71a2f845446ca6355bd2ba6c8ddc'
            },
            data=json.dumps({ 'documents': chunks })
        )
        try:
            r.raise_for_status()
        except:
            print('Error for ' + path)
        
        response = r.json().get('documents', [])
        for doc in response:
            identifier, part = doc['id'].split('.')
            part = int(part)
            couchdb_url = 'http://148.251.91.133:5984/parlasentiment3/{}.{}'.format(identifier, part)
            doc['sentiment'] = sentiments[identifier][int(part)]
            doc['raw'] = ' '.join(words[(part * per_batch):(part * per_batch + per_batch)])
            r = requests.put(couchdb_url, data=json.dumps(doc))
            r.raise_for_status()
    print('Finished ' + path)

ChunkedEncodingError: ("Connection broken: ConnectionResetError(54, 'Connection reset by peer')", ConnectionResetError(54, 'Connection reset by peer'))